In [1]:
# In this notebook, we will calculate the change in atmospheric heat flux convergence (including the stratosphere)

# By: Ty Janoski
# edited: 05.30.20

In [2]:
# import statements
import xarray as xr
import numpy as np
import dask.array as da
import xesmf as xe
from dask.diagnostics import ProgressBar
import glob

In [4]:
# start by reading in the water vapor kernel
q_kern = xr.open_dataset('/dx05/tylerj/d10/Arctic_Research/CMIP5_Arctic_Amplification/\
CAM5_kernels/q.kernel.plev.nc')
lats = q_kern.lat
lons = q_kern.lon
# some models have higher vertical resolution than the kernels. We only need 17 levels
plevs = [float(p) for p in q_kern.plev]

# create a dataset with lat/lons for regridding
ds_out = xr.Dataset({'lat': (['lat'], lats),
                     'lon': (['lon'], lons),
                    }
                   )

In [5]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR','GFDL-ESM2G',
          'FGOALS-g2', 'bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
for mod in models:
    print(mod)
    # rlus for BNU-ESM is cut off, so we will use the last 50 years offset by 1 year
    if(mod=='BNU-ESM'):
        start = -612
        end = -12
    else:
        start = -600
        end = None
    # read in variables
    if mod in ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2']:
        # read in air temperature from last 50 years of piControl
        ta = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/ta_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',
                              use_cftime=True).ta.isel(
            time=slice(start,end)).sel(plev=plevs)
        hus = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/hus_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',
                              use_cftime=True).hus.isel(
            time=slice(start,end)).sel(plev=plevs)
        ua = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/ua_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',
                              use_cftime=True).ua.isel(
            time=slice(start,end)).sel(plev=plevs)
        va = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/va_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',
                              use_cftime=True).va.isel(
            time=slice(start,end)).sel(plev=plevs)
    else:
        path = glob.glob('/dx07/tylerj/CMIP5_output/piControl/ta_Amon_'+mod+'*.nc')[0]
        ta = xr.open_dataset(path,use_cftime=True).ta.isel(
            time=slice(start,end)).sel(plev=plevs)
        path = glob.glob('/dx07/tylerj/CMIP5_output/piControl/hus_Amon_'+mod+'*.nc')[0]
        hus = xr.open_dataset(path,use_cftime=True).hus.isel(
            time=slice(start,end)).sel(plev=plevs)
        path = glob.glob('/dx07/tylerj/CMIP5_output/piControl/ua_Amon_'+mod+'*.nc')[0]
        ua = xr.open_dataset(path,use_cftime=True).ua.isel(
            time=slice(start,end)).sel(plev=plevs)
        path = glob.glob('/dx07/tylerj/CMIP5_output/piControl/va_Amon_'+mod+'*.nc')[0]
        va = xr.open_dataset(path,use_cftime=True).va.isel(
            time=slice(start,end)).sel(plev=plevs)
    
    # we also need orography
    orog = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/orog_fx_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).orog
    if mod in ['ACCESS1-0','ACCESS1-3']:
        regridder = xe.Regridder(orog,ta,'bilinear',periodic=True,reuse_weights=True)
        orog = regridder(orog.compute())
    orog = xr.broadcast(orog,ta)[0]
    
    # TE = total energy
    with ProgressBar():
        TE = ((ta * 1004) + (hus * 2.5e6) + orog*9.81 + (ua**2 + va**2)*0.5).compute()
    
    # regrid TE to CAM5 resolution
    regridder = xe.Regridder(TE,ds_out,'bilinear',periodic=True,reuse_weights=True)
    TE = regridder(TE)
    
    # read in dz from "dz_strat_included.ipynb"
    dz = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_dz_strat_pic.nc',
                        use_cftime=True)
    
    # vertically integrate, differentiate with respect to time in seconds
    TE_vert = (dz * TE).sum(dim='plev')/9.81
    dEdt = TE_vert.differentiate(coord='time',edge_order=2,datetime_unit='s')
    
    rsds = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rsds_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rsds.isel(
        time=slice(start,end))
    if(mod == 'CCSM4'):
        rsus = np.array(xr.open_dataset('http://strega.ldeo.columbia.edu:81/CMIP5/.byScenario/.piControl/' + 
                            '.atmos/.mon/.rsus/.CCSM4/.r1i1p1/.rsus/dods',
                               decode_times=False)['rsus'].isel(T=slice(start,end)))
    else:
        rsus = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rsus_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',
                              use_cftime=True).rsus.isel(
        time=slice(start,end))
    rlds = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rlds_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rlds.isel(
        time=slice(start,end))
    if(mod=='BNU-ESM'):
        rlus = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rlus_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',
                              use_cftime=True).rlus.isel(
            time=slice(start+6,end+6))
    else:
        rlus = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rlus_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',
                              use_cftime=True).rlus.isel(
            time=slice(start,end))
    hfls = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/hfls_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).hfls.isel(
        time=slice(start,end)).fillna(0)
    hfss = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/hfss_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).hfss.isel(
        time=slice(start,end)).fillna(0)
    rsut = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rsut_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rsut.isel(
        time=slice(start,end))
    rlut = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rlut_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rlut.isel(
        time=slice(start,end))
    rsdt = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rsdt_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rsdt.isel(
        time=slice(start,end))

    if mod in ['ACCESS1-0','ACCESS1-3']:
        regridder = xe.Regridder(rsds,ds_out,'bilinear',periodic=True,reuse_weights=True)
    Fsfc = rsds - rsus - rlus + rlds - hfls - hfss
    Fsfc = regridder(Fsfc)
    
    Ftoa = rsdt - rsut - rlut
    Ftoa = regridder(Ftoa)
    
    with ProgressBar():
        Fconv = (Fsfc - Ftoa + dEdt).compute()
    print(Fconv)
    clim = Fconv.dz.groupby(Fconv.time.dt.month).mean(dim='time')
    
    # -----------------------------------------------------------------------------------------
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    # -----------------------------------------------------------------------------------------
    # we gada do the same for abrupt4xCO2
    start = None
    end = 1800
    # read in variables
    if mod in ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2']:
        # read in air temperature from last 50 years of piControl
        ta = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/ta_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',
                              use_cftime=True).ta.isel(
            time=slice(start,end)).sel(plev=plevs)
        hus = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/hus_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',
                              use_cftime=True).hus.isel(
            time=slice(start,end)).sel(plev=plevs)
        ua = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/ua_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',
                              use_cftime=True).ua.isel(
            time=slice(start,end)).sel(plev=plevs)
        va = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/va_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',
                              use_cftime=True).va.isel(
            time=slice(start,end)).sel(plev=plevs)
    else:
        path = glob.glob('/dx07/tylerj/CMIP5_output/abrupt4xCO2/ta_Amon_'+mod+'*.nc')[0]
        ta = xr.open_dataset(path,use_cftime=True).ta.isel(
            time=slice(start,end)).sel(plev=plevs)
        path = glob.glob('/dx07/tylerj/CMIP5_output/abrupt4xCO2/hus_Amon_'+mod+'*.nc')[0]
        hus = xr.open_dataset(path,use_cftime=True).hus.isel(
            time=slice(start,end)).sel(plev=plevs)
        path = glob.glob('/dx07/tylerj/CMIP5_output/abrupt4xCO2/ua_Amon_'+mod+'*.nc')[0]
        ua = xr.open_dataset(path,use_cftime=True).ua.isel(
            time=slice(start,end)).sel(plev=plevs)
        path = glob.glob('/dx07/tylerj/CMIP5_output/abrupt4xCO2/va_Amon_'+mod+'*.nc')[0]
        va = xr.open_dataset(path,use_cftime=True).va.isel(
            time=slice(start,end)).sel(plev=plevs)
    
    # we also need orography
    orog = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/orog_fx_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).orog
    if mod in ['ACCESS1-0','ACCESS1-3']:
        regridder = xe.Regridder(orog,ta,'bilinear',periodic=True,reuse_weights=True)
        orog = regridder(orog.compute())
    orog = xr.broadcast(orog,ta)[0]
    
    # TE = total energy
    with ProgressBar():
        TE = ((ta * 1004) + (hus * 2.5e6) + orog*9.81 + (ua**2 + va**2)*0.5).compute()
    
    # regrid TE to CAM5 resolution
    regridder = xe.Regridder(TE,ds_out,'bilinear',periodic=True,reuse_weights=True)
    TE = regridder(TE)
    
    # read in dz from "dz_strat_included.ipynb"
    dz = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_dz_strat_4xCO2.nc',
                        use_cftime=True)
    
    # vertically integrate, differentiate with respect to time in seconds
    TE_vert = (dz * TE).sum(dim='plev')/9.81
    dEdt = TE_vert.differentiate(coord='time',edge_order=2,datetime_unit='s')
    
    rsds = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/rsds_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rsds.isel(
        time=slice(start,end))
    rsus = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/rsus_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rsus.isel(
        time=slice(start,end))
    rlds = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/rlds_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rlds.isel(
        time=slice(start,end))
    rlus = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/rlus_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rlus.isel(
        time=slice(start,end))
    hfls = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/hfls_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).hfls.isel(
        time=slice(start,end)).fillna(0)
    hfss = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/hfss_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).hfss.isel(
        time=slice(start,end)).fillna(0)
    rsut = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/rsut_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rsut.isel(
        time=slice(start,end))
    rlut = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/rlut_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rlut.isel(
        time=slice(start,end))
    rsdt = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/rsdt_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',
                          use_cftime=True).rsdt.isel(
        time=slice(start,end))

    if mod in ['ACCESS1-0','ACCESS1-3']:
        regridder = xe.Regridder(rsds,ds_out,'bilinear',periodic=True,reuse_weights=True)
    Fsfc = rsds - rsus - rlus + rlds - hfls - hfss
    Fsfc = regridder(Fsfc)
    
    Ftoa = rsdt - rsut - rlut
    Ftoa = regridder(Ftoa)
    
    with ProgressBar():
        Fconv = (Fsfc - Ftoa + dEdt).compute()
    
    print(Fconv)
        
    diff = Fconv.dz - da.tile(clim,(150,1,1))
    with ProgressBar():
        diff = diff.compute()
    diff.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+
                                           mod+'_atmos_conv.nc')
    
    del diff, dz, ta, hus, ua, va, TE, orog

CCSM4
[########################################] | 100% Completed |  5.6s
Reuse existing file: bilinear_192x288_192x288_peri.nc
[########################################] | 100% Completed |  7.8s
<xarray.Dataset>
Dimensions:  (lat: 192, lon: 288, time: 600)
Coordinates:
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * time     (time) object 1251-01-16 12:00:00 ... 1300-12-16 12:00:00
Data variables:
    dz       (time, lat, lon) float64 41.22 41.22 41.29 ... 223.5 223.5 223.5
[########################################] | 100% Completed |  1min 24.8s
Reuse existing file: bilinear_192x288_192x288_peri.nc
[########################################] | 100% Completed | 25.3s
<xarray.Dataset>
Dimensions:  (lat: 192, lon: 288, time: 1800)
Coordinates:
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ...

In [5]:
def spatial_mean(ds,lat_bound=-90):
    """Take the mean over latitude and longitude dimensions,
    while weighting for latitude.

    PARAMETERS
    ----------
    datasets : list of xarray.Dataset
        A list of xarray datasets to return .5

    RETURNS
    -------
    filelist : A list of xarray datasets retrieved from server.
    """
    area_of_int = ds.mean(dim='lon').sel(lat = slice(lat_bound, None))
    weights = (np.cos(np.deg2rad(area_of_int['lat']))/
               np.sum(np.cos(np.deg2rad(area_of_int['lat']))))
    return((area_of_int * weights).sum(dim='lat'))